In [1]:
from flask import Flask, render_template, request, redirect, url_for
import random
from faker import Faker
from datetime import datetime
from jinja2 import DictLoader

app = Flask(__name__)
fake = Faker()

# Generate fake data
properties = [{
    "id": i,
    "address": fake.street_address(),
    "city": fake.city(),
    "roof_type": random.choice(["Tile", "Shingle", "Metal"]),
    "last_roof_date": fake.date_between(start_date='-30y', end_date='today').strftime('%Y-%m-%d'),
    "owner": fake.name(),
    "parcel_name": fake.company(),
    "llc_mailing_address": fake.address(),
    "contact_info": [],
    "notes": ""
} for i in range(1, 51)]

# Enhanced and Styled HTML Templates in DictLoader
app.jinja_loader = DictLoader({
    'index.html': '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>SCI Roofing Permits</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
        <style>
            body { margin: 20px; }
            table { width: 100%; }
            th, td { text-align: center; vertical-align: middle; }
            th input, th select { width: 90%; margin: auto; }
            img { max-width: 250px; margin-bottom: 20px; }
        </style>
        <script>
            function toggleDateInputs() {
                var filter = document.getElementById("date_filter").value;
                document.getElementById("date_to").style.display = filter === "between" ? "inline-block" : "none";
            }
        </script>
    </head>
    <body onload="toggleDateInputs()">
        <div class="container">
            <img src="{{ url_for('static', filename='SCILOGO.png') }}">
            <h2 class="mb-4">SCI Roofing Permits</h2>
            <form method="get" class="mb-4 row g-2">
                <div class="col-md-3">
                    <input type="text" class="form-control" name="address" placeholder="Search Address or City" value="{{request.args.get('address', '')}}">
                </div>
                <div class="col-md-2">
                    <input type="text" class="form-control" name="roof_type" placeholder="Roof Type" value="{{request.args.get('roof_type', '')}}">
                </div>
                <div class="col-md-2">
                    <input type="text" class="form-control" name="owner" placeholder="Owner" value="{{request.args.get('owner', '')}}">
                </div>
                <div class="col-md-3">
                    <select class="form-select" name="date_filter" id="date_filter" onchange="toggleDateInputs()">
                        <option value="">Date Filter</option>
                        <option value="before" {% if request.args.get('date_filter') == 'before' %}selected{% endif %}>Before</option>
                        <option value="after" {% if request.args.get('date_filter') == 'after' %}selected{% endif %}>After</option>
                        <option value="between" {% if request.args.get('date_filter') == 'between' %}selected{% endif %}>Between</option>
                    </select>
                </div>
                <div class="col-md-2">
                    <input type="date" class="form-control" name="date_from" value="{{request.args.get('date_from', '')}}">
                    <input type="date" class="form-control" name="date_to" id="date_to" value="{{request.args.get('date_to', '')}}" style="display:none; margin-top: 5px;">
                </div>
                <div class="col-md-12 text-end">
                    <button class="btn btn-primary">Search</button>
                </div>
            </form>
            <table class="table table-striped table-bordered table-hover">
                <thead class="table-dark">
                    <tr>
                        <th>Property Address</th>
                        <th>City</th>
                        <th>Roof Type</th>
                        <th>Date of Last Roof Permit</th>
                        <th>Owner</th>
                    </tr>
                </thead>
                <tbody>
                    {% for prop in properties %}
                    <tr onclick="window.location.href='{{ url_for('edit_property', prop_id=prop.id) }}'">
                        <td>{{prop.address}}</td>
                        <td>{{prop.city}}</td>
                        <td>{{prop.roof_type}}</td>
                        <td>{{prop.last_roof_date}}</td>
                        <td>{{prop.owner}}</td>
                    </tr>
                    {% endfor %}
                </tbody>
            </table>
        </div>
    </body>
    </html>
    ''',
    'edit_property.html': '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Edit Property</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
        <style>
            body { padding: 30px; }
        </style>
    </head>
    <body>
        <div class="container">
            <h2>Edit Property Details</h2>
            <form method="post" class="mt-4">
                <div class="mb-2"><label>Address</label><input class="form-control" type="text" name="address" value="{{prop.address}}"></div>
                <div class="mb-2"><label>Roof Type</label><input class="form-control" type="text" name="roof_type" value="{{prop.roof_type}}"></div>
                <div class="mb-2"><label>Last Roof Date</label><input class="form-control" type="date" name="last_roof_date" value="{{prop.last_roof_date}}"></div>
                <div class="mb-2"><label>Owner</label><input class="form-control" type="text" name="owner" value="{{prop.owner}}"></div>
                <div class="mb-2"><label>Parcel Name</label><input class="form-control" type="text" name="parcel_name" value="{{prop.parcel_name}}"></div>
                <div class="mb-2"><label>LLC Mailing Address</label><input class="form-control" type="text" name="llc_mailing_address" value="{{prop.llc_mailing_address}}"></div>
                <div class="mb-2"><label>Notes</label><textarea class="form-control" name="notes">{{prop.notes}}</textarea></div>
                <button class="btn btn-success mt-2">Save</button>
            </form>
        </div>
    </body>
    </html>
    '''
})
@app.route('/', methods=['GET', 'POST'])
def index():
    address = request.args.get('address', '').lower()
    roof_type = request.args.get('roof_type', '').lower()
    owner = request.args.get('owner', '').lower()
    date_filter = request.args.get('date_filter', '')
    date_from = request.args.get('date_from', '')
    date_to = request.args.get('date_to', '')

    filtered_properties = [prop for prop in properties if
                           address in prop['address'].lower() or address in prop['city'].lower()]

    if roof_type:
        filtered_properties = [prop for prop in filtered_properties if roof_type in prop['roof_type'].lower()]
    if owner:
        filtered_properties = [prop for prop in filtered_properties if owner in prop['owner'].lower()]

    if date_filter and date_from:
        date_from_obj = datetime.strptime(date_from, '%Y-%m-%d')
        if date_filter == 'before':
            filtered_properties = [prop for prop in filtered_properties if datetime.strptime(prop['last_roof_date'], '%Y-%m-%d') < date_from_obj]
        elif date_filter == 'after':
            filtered_properties = [prop for prop in filtered_properties if datetime.strptime(prop['last_roof_date'], '%Y-%m-%d') > date_from_obj]
        elif date_filter == 'between' and date_to:
            date_to_obj = datetime.strptime(date_to, '%Y-%m-%d')
            filtered_properties = [prop for prop in filtered_properties if date_from_obj <= datetime.strptime(prop['last_roof_date'], '%Y-%m-%d') <= date_to_obj]

    return render_template('index.html', properties=filtered_properties)

@app.route('/property/<int:prop_id>', methods=['GET', 'POST'])
def edit_property(prop_id):
    prop = next((item for item in properties if item['id'] == prop_id), None)
    if request.method == 'POST':
        prop['address'] = request.form['address']
        prop['roof_type'] = request.form['roof_type']
        prop['last_roof_date'] = request.form['last_roof_date']
        prop['owner'] = request.form['owner']
        prop['parcel_name'] = request.form['parcel_name']
        prop['llc_mailing_address'] = request.form['llc_mailing_address']
        prop['notes'] = request.form['notes']

        return redirect(url_for('index'))

    return render_template('edit_property.html', prop=prop)

if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Aug/2025 12:17:33] "GET /property/1 HTTP/1.1" 200 -
